## Notebook di training

In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder

### Estrazione del dataset di training dal file train.csv

In [3]:
# Estraggo dal file train.csv il dataset di training corrispondente
train_path_file = './train.csv'
X_train_set = pd.read_csv(train_path_file)

# Rimuovo le feature che sicuramente non danno alcun contributo al training
X_train_set.drop(['PatientId'], axis=1, inplace=True)
X_train_set.drop(['AppointmentID'], axis=1, inplace=True)

# Elimino le righe che hanno il campo 'No-show' mancante
X_train_set.dropna(axis=0, subset=['No-show'], inplace=True)

# Effettuo il label encoding delle variabili categoriche
label = LabelEncoder()

for features in X_train_set:
      X_train_set[features]=label.fit_transform(X_train_set[features])

#Mostro il train set estratto
X_train_set

,No-show,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0,0,17205,26,42,75,0,0,1,0,0,1
1,0,1,62569,22,31,48,0,0,0,0,0,0
2,1,1,20492,8,47,75,0,1,0,0,0,1
3,0,0,32576,7,33,23,0,0,0,0,0,0
4,0,0,18421,1,7,26,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
70732,0,0,4284,10,39,43,0,0,0,0,0,0
70733,0,1,20090,1,49,8,0,1,0,1,0,0
70734,0,0,59703,21,19,29,1,0,0,0,0,0
70735,1,0,40231,15,29,77,0,0,0,0,0,0


#### Separo il target dalle features. Procedo poi con l'abbattimento dei pesi servendomi di una Ridge Regression. Essa possiede un fattore di regolarizzazione che abbatte i valori dei pesi piu grandi (ne fa il quadrato e li aggiunge alla Loss) mentre lascia passare quelli piccoli ( quelli tra 0 e 1 vengono difatti ridotti con il quadrato) sempre usando una discesa del gradiente. Di seguito la formula:

$$R(\boldsymbol{w}) = \sum_l  w_l^2 $$

In [4]:
from sklearn.linear_model import Ridge

# Rilevo il target di training
y_train = X_train_set['No-show']

# Rilevo tutte le features di training
X_train_full = X_train_set.drop(['No-show'], axis=1)

Ridge= Ridge(alpha =0.1,random_state = 0, max_iter = 100).fit(X_train_full, y_train)
Ridge_model = SelectFromModel(Ridge, prefit =True) 

features_sel_Ridge = X_train_full.columns[Ridge_model.get_support()]

X_train = X_train_full[features_sel_Ridge].copy()

# Mostro le features di training
X_train

,Scholarship,Diabetes,Alcoholism,Handcap,SMS_received
0,0,1,0,0,1
1,0,0,0,0,0
2,0,0,0,0,1
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
70732,0,0,0,0,0
70733,0,0,1,0,0
70734,1,0,0,0,0
70735,0,0,0,0,0


#### Configuro le funzionalità che saranno associate alla Pipeline. Nello specifico riguardano  il modello di classificazione e la gestione dei valori mancanti. Questi ultimi sono risolti, per imputazione, con la SimpleImputer con il parametro 'most_fruquent'. Tale strategia prevede la sostituzione di tutti i valori mancanti con il valore della colonna più ricorrente. Effettuando varie prove, il parametro 'most_frequent' si è rivelato il migliore. Per quanto riguarda invece il modello di classificazione, ho optato per il LogisticRegression

In [5]:
from sklearn.preprocessing import StandardScaler

# Processo per i dati numerici
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Costruzione del processo per i dati categorici e per i dati numerici
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, X_train.columns)
])

# Scelta come modello della LogisticRegression
model = LogisticRegression(random_state=0)

#### Costruisco la Pipeline sulla base delle configurazioni sopra descritte e avvio l'addestramento sui dati di training

In [6]:
# Definizion della Pipeline con configurazioni e modelli predisposti nella fase precedente
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Avvia l'addestramento
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='most_frequent',
                                                                verbose=0),
                                                  Index(['Scholarship', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received'], dtype='object'))],
                                   verbose=False)),
                

### Estrazione del dataset di test dal file test.csv

In [7]:
# Estraggo dal file test.csv il Dataset di test corrispondente
test_path_file = './test.csv'
X_valid_set = pd.read_csv(test_path_file)

# Rimuovo le feature che sicuramente non danno alcun contributo
X_valid_set.drop(['PatientId'], axis=1, inplace=True)
X_valid_set.drop(['AppointmentID'], axis=1, inplace=True)

# Elimino le righe che hanno il campo No-show mancante
X_valid_set.dropna(axis=0, subset=['No-show'], inplace=True)

# Effettuo il label encoding delle variabili categoriche
label = LabelEncoder()

for features in X_valid_set:
    X_valid_set[features]=label.fit_transform(X_valid_set[features])

# Mostra il test set estratto
X_valid_set

,No-show,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0,1,10124,13,62,24,0,0,0,0,0,0
1,0,0,8868,25,19,35,0,0,0,0,0,1
2,1,0,6357,7,25,73,0,0,0,0,0,1
3,1,0,16270,23,19,27,1,0,0,0,0,0
4,0,0,6316,7,58,42,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17679,1,0,3692,1,68,40,0,1,1,0,0,1
17680,0,0,4104,0,14,36,0,0,0,0,0,0
17681,0,0,120,3,60,36,0,0,0,0,0,1
17682,0,0,12121,18,9,54,0,0,0,0,0,1


### Suddivisione tra target e features e prendo l'insieme ottimale di queste ultime individuate nel training set tramite Ridge

In [8]:
# Rilevo il target di test
y_valid = X_valid_set['No-show']

# Rilevo tutte le features di test
X_valid_full = X_valid_set.drop(['No-show'], axis=1)

# Popolo il test set con l'insieme ottimale di features individuate nel training set
X_valid = X_valid_full[features_sel_Ridge].copy()

# Mostra le feature di test
X_valid

,Scholarship,Diabetes,Alcoholism,Handcap,SMS_received
0,0,0,0,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1
...,...,...,...,...,...
17679,0,1,0,0,1
17680,0,0,0,0,0
17681,0,0,0,0,1
17682,0,0,0,0,1


### Verifico l'accuracy del modello

In [9]:
# Effettua la previsione
preds = pipeline.predict(X_valid)

test_acc=metrics.accuracy_score(y_valid, preds)
print('Accuracy test: %s'  % test_acc)

Accuracy test: 0.7995363040036191
